In [1]:
! nvidia-smi

Tue Apr 19 20:04:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:61:00.0 Off |                    0 |
| N/A   32C    P0    24W / 250W |      6MiB / 12198MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:DB:00.0 Off |                    0 |
| N/A   

In [2]:
import os, re, math, copy, time, sys
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
# from tqdm import tqdm
import tqdm

from transformers import BertForPreTraining, BertModel, BertConfig, BertTokenizer
from transformers import BertTokenizer, BertForSequenceClassification
from tokenizers import ByteLevelBPETokenizer

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from scipy.special import softmax
import random

In [4]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [5]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [6]:
MAX_LEN=128

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [8]:
import torch
import torch.nn as nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, freeze_bert=False):

        super(BertClassifier, self).__init__()

        self.PAD = tokenizer.pad_token_id
        # Instantiate BERT model
        self.bert = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=3)

        
    def forward(self, C, c_labels):

        output = self.bert(
            input_ids = C, 
            attention_mask = (C!=self.PAD),
            labels = c_labels
        )   
        

        return output

In [9]:
bert_classifier = BertClassifier()
bert_classifier.load_state_dict(torch.load('bert-parameters.pt'))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

<All keys matched successfully>

In [10]:
qa = pd.read_csv("QA.csv")

In [11]:
qa

,SNo,Question,Gold standard answer,Dummy answer,Unnamed: 4
0,1.0,What is deep learning?,Deep learning is a part of machine learning wi...,Profound learning may be a portion of machine ...,NaN
1,2.0,"What are the main differences between AI, Mach...",AI stands for Artificial Intelligence. It is a...,AI stands for Manufactured Insights. It may be...,NaN
2,3.0,Differentiate supervised and unsupervised deep...,Supervised learning is a system in which both ...,Administered learning could be a framework in ...,NaN
3,4.0,Do you think that deep network is better than ...,Both shallow and deep networks are good enough...,Both shallow and deep networks are capable of ...,NaN
4,5.0,"What do you mean by ""overfitting""?",Overfitting is the most common issue which occ...,It ordinarily happens when a profound learning...,NaN
...,...,...,...,...,...
107,NaN,NaN,NaN,NaN,NaN
108,NaN,NaN,NaN,NaN,NaN
109,NaN,NaN,NaN,NaN,NaN
110,NaN,NaN,NaN,NaN,NaN


In [12]:
qa.drop(["Unnamed: 4"], axis = 1, inplace=True)

In [13]:
qa.dropna(axis=0, how='any', inplace=True)

In [14]:
qa.drop(["SNo"], axis = 1, inplace=True)

In [15]:
qa = qa.reset_index()

In [16]:
qa.drop(["index"], axis = 1, inplace=True)

In [17]:
qa

,Question,Gold standard answer,Dummy answer
0,What is deep learning?,Deep learning is a part of machine learning wi...,Profound learning may be a portion of machine ...
1,"What are the main differences between AI, Mach...",AI stands for Artificial Intelligence. It is a...,AI stands for Manufactured Insights. It may be...
2,Differentiate supervised and unsupervised deep...,Supervised learning is a system in which both ...,Administered learning could be a framework in ...
3,Do you think that deep network is better than ...,Both shallow and deep networks are good enough...,Both shallow and deep networks are capable of ...
4,"What do you mean by ""overfitting""?",Overfitting is the most common issue which occ...,It ordinarily happens when a profound learning...
...,...,...,...
93,What is the need to add randomness in the weig...,"If you set the weights to zero, then every neu...",We need to add randomness to the weights to en...
94,How can you train hyperparameters in a neural ...,Hyperparameters in a neural network can be tra...,Hyperparameters in a neural can be trained usi...
95,What do you mean by end-to-end learning?,It's a deep learning procedure in which a mode...,End to end learning is learning a model which ...
96,What do you understand about gradient clipping...,Gradient Clipping is a technique for dealing w...,Gradient Clipping is the process of cipping th...


In [18]:
import re

class data():
      
  def __init__(self, max_len):
    super(data, self).__init__()
    self.data=[]

    premise = qa["Gold standard answer"].tolist()
    hypo = qa["Dummy answer"].tolist()
    length = len(premise)

    for i in tqdm.auto.tqdm(range(length)):
        
        premise_cleaned=premise[i].replace("\n","")
        premise_cleaned=re.sub(' +',' ',premise_cleaned)
        premise_cleaned=premise_cleaned.lower()

        hypo_cleaned=hypo[i].replace("\n","")
        hypo_cleaned=re.sub(' +',' ',hypo_cleaned)
        hypo_cleaned=hypo_cleaned.lower()

        sent = "[CLS]" + premise_cleaned + "[SEP]" + hypo_cleaned + "[SEP]"
        enc_sent = tokenizer(sent, padding='max_length', add_special_tokens=False)['input_ids']
        self.data.append(torch.tensor(enc_sent[:max_len]))

  def __getitem__(self, index):
    return self.data[index]
  def __len__(self):
    return len(self.data)

In [19]:
qa_data = data(MAX_LEN)

In [20]:
bert_classifier.eval()

BertClassifier(
  (bert): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=Tru

In [21]:
prob_list = []
for i in range(len(qa_data)):
  output = bert_classifier(torch.unsqueeze(qa_data[i],0),None)
  outputs = output[0].detach().cpu().numpy()
  prob = softmax(outputs)
  prob_list.append(prob)

In [22]:
premise = qa["Gold standard answer"].tolist()
hypo = qa["Dummy answer"].tolist()

In [23]:
with open('qa_results.tsv', 'w') as f:
    f.write("S.No."+"\t"+ "Gold Standard Answer" + "\t" +"Student Answer" + "\t" +"Prob for Entailment" + "\t" + "Prob for Neutral" + "\t" + "Prob for Contradiction" + "\n")
    for i in range(len(premise)):
        premise_cleaned=premise[i].replace("\n","")
        premise_cleaned=re.sub(' +',' ',premise_cleaned)
        premise_cleaned=premise_cleaned.lower()

        hypo_cleaned=hypo[i].replace("\n","")
        hypo_cleaned=re.sub(' +',' ',hypo_cleaned)
        hypo_cleaned=hypo_cleaned.lower()

        f.write(str(i) + "\t" + premise_cleaned + "\t" + hypo_cleaned + "\t" + str(prob_list[i][0][0]) + "\t" + str(prob_list[i][0][1]) + "\t" + str(prob_list[i][0][2]) + "\n")